# Visualization of backtests

This notebook can be used for the visualization of backtest results for the comparison of pure market making strategies and the trained AIRL strategy.

In [ ]:
import os
import pickle
import time

import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from datetime import datetime

from lob.backtest_metrics import drawdowns
from lob.commissions import BitCommissions
from lob.exchange import Exchange
from lob.traders import PureMarketMaker
from lob.plots import (
    set_plot_style,
    visualize_backtest,
    make_drawdown_plot,
    make_plot
)
from lob.utils import get_lot_size, get_tick_size, ensure_dir_exists
from rl.utils import send_notification

In [ ]:
# Configure Polars 
cfg = pl.Config()
cfg.set_tbl_rows(20)

# Configure plotting
set_plot_style()

In [ ]:
# Define custom colors
color_green = "#13961a"
color_red = "#eb5c14"

In [ ]:
# Set random seed
SEED = 1

In [ ]:
save_dir = "images"
ensure_dir_exists(save_dir)

### Visualize backtests for PMM with priority 1 and 100 volume

In [ ]:
save_path = "automated_backtests/results_2024-02-25_10-58-39.pickle"

In [ ]:
# Load the results from a pickle file
with open(save_path, "rb") as handle:
    results = pickle.load(handle)

In [ ]:
### PMM priority 1 volume 100

# Load the results
ts = results["PMM_prior_1_vol_100"]["timestamps"]
trader_stats = results["PMM_prior_1_vol_100"]["trader_stats"]
initial_cost = results["PMM_prior_1_vol_100"]["initial_cost"]

# Plot the results
# ----------------------------------------------------------------------------
# PLOT - Adjusted PnL
make_plot(
    x=ts,
    y=trader_stats["adj_pnl"],
    xlabel="Time",
    ylabel="P&L (USDT)",
    save_path=os.path.join(save_dir, "PMM_pnl_100.pdf")
)
print(f"Final P&L: {trader_stats['adj_pnl'][-1]}")

# PLOT - Returns
equity = pd.Series(np.array(trader_stats["adj_pnl"]) + initial_cost)
returns = equity.pct_change() * 100
make_plot(
    x=ts,
    y=returns,
    xlabel="Time",
    ylabel="Returns (%)",
    save_path=os.path.join(save_dir, "PMM_returns_100.pdf")
)
print("Returns stats")
print(returns.describe())

# PLOT - Drawdowns
dd = drawdowns(equity)
make_drawdown_plot(
    x=ts,
    y=dd,
    xlabel="Time",
    ylabel="Drawdown (%)",
    save_path=os.path.join(save_dir, "PMM_drawdowns_100.pdf")
)
print("Drawdown stats")
print(dd.describe())

# PLOT - Inventory
make_plot(
    x=ts,
    y=trader_stats["inventory"],
    xlabel="Time",
    ylabel="Inventory (SOL)",
    color="darkorange",
    save_path=os.path.join(save_dir, "PMM_inventory_100.pdf")
)
print("Inventory stats")
print(pd.Series(trader_stats["inventory"]).describe())

# PLOT - Total traded volume
make_plot(
    x=ts,
    y=trader_stats["total_volume"],
    xlabel="Time",
    ylabel="Traded volume (USDT)",
    ylim=(-40000, 840000),
    color="darkorange",
    save_path=os.path.join(save_dir, "PMM_volume_100.pdf")
)
print("Total volume: ", trader_stats["total_volume"][-1])

# PLOT - Transaction costs
make_plot(
    x=ts,
    y=trader_stats["cum_costs"],
    xlabel="Time",
    ylabel="Transaction fees (USDT)",
    ylim=(-20, 420),
    color="red",
    save_path=os.path.join(save_dir, "PMM_fees_100.pdf")
)
print("Total fees: ", trader_stats["cum_costs"][-1])

In [ ]:
from scipy.stats import gaussian_kde

In [ ]:
returns.dropna()

In [ ]:
# Histogram of returns with KDE
fig = plt.figure(figsize=(12, 4.5))
plt.hist(returns, bins=50, alpha=1, log=True)
# Add kernel density estimate
plt.xlabel("Returns (%)")
plt.ylabel("Frequency")
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "PMM_returns_hist_100.pdf"))

### Comparison of PMM strategies with volume 100

In [ ]:
save_path = "automated_backtests/results_2024-02-25_10-58-39.pickle"

In [ ]:
# Load the results from a pickle file
with open(save_path, "rb") as handle:
    results = pickle.load(handle)

In [ ]:
results.keys()

In [ ]:
# PnL plot 

# Set parameters
figsize = (12, 4.5)
xlabel = "Time"
ylabel = "P&L (USDT)"

plt.figure(figsize=figsize)
i = 0
for key, value in results.items():
    x = value["timestamps"]
    y = value["trader_stats"]["adj_pnl"]
    label = f"PMM (priority {i})"
    plt.plot(x, y, label=label)
    print(f"{key} - {y[-1]:.2f}")
    i += 1
    
# plt.plot(x, y, color=color)
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "PMM_comparison_pnl_100.pdf"))
plt.show()

In [ ]:
# Set parameters
figsize = (12, 4.5)
xlabel = "Time"
ylabel = "Traded volume (USDT)"

plt.figure(figsize=figsize)
i = 0
for key, value in results.items():
    x = value["timestamps"]
    y = value["trader_stats"]["total_volume"]
    label = f"PMM (priority {i})"
    plt.plot(x, y, label=label)
    print(f"{key} - {y[-1]:.2f}")
    i += 1
    
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "PMM_comparison_volume_100.pdf"))
plt.show()

In [ ]:
# Compute hitting probability
for key, value in results.items():
    trades = np.array(value["trader_stats"]["trade_count"])
    hits = np.where(trades > 0, 1, 0)
            
    print(f"{key} - {np.mean(hits)*100:.2f}")
    print()

In [ ]:
index = 0

# Spread histogram
fig = plt.figure(figsize=(12, 4.5))
for key, value in results.items():
    spread = np.array(value["trader_stats"]["quoted_ask_price"]) - np.array(value["trader_stats"]["quoted_bid_price"])
    plt.hist(spread, bins=50, alpha=0.75, log=False, label=f"PMM (priority {index})")
    mean = np.mean(spread)
    plt.vlines(mean, 0, 50000, color=f"C{index}", linestyle="--")
    print(f"{key} - mean: {mean:.4f}")
    index += 1
# Add kernel density estimate
plt.xlabel("Spread (USDT)")
plt.ylabel("Count")
# Cut off the outliers
plt.xlim(0, 0.24)
plt.xticks(np.arange(0, 0.24, 0.01), rotation=45)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "PMM_comparison_spread_100.pdf"))
plt.show()

### Comparison of PMM strategies with volume 10

In [ ]:
save_path = "automated_backtests/results_2024-02-25_13-08-03.pickle"

In [ ]:
# Load the results from a pickle file
with open(save_path, "rb") as handle:
    results = pickle.load(handle)

In [ ]:
results.keys()

In [ ]:
# PnL plot 

# Set parameters
figsize = (12, 4.5)
xlabel = "Time"
ylabel = "P&L (USDT)"


plt.figure(figsize=figsize)
i = 0
for key, value in results.items():
    x = value["timestamps"]
    y = value["trader_stats"]["adj_pnl"]
    label = f"PMM (priority {i})"
    plt.plot(x, y, label=label)
    print(f"{key} - {y[-1]:.2f}")
    i += 1
# plt.plot(x, y, color=color)
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.ylim(-105, 55)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "PMM_comparison_pnl_10.pdf"))
plt.show()

In [ ]:
# Set parameters
figsize = (12, 4.5)
xlabel = "Time"
ylabel = "Traded volume (USDT)"


plt.figure(figsize=figsize)
i = 0
for key, value in results.items():
    x = value["timestamps"]
    y = value["trader_stats"]["total_volume"]
    label = f"PMM (priority {i})"
    plt.plot(x, y, label=label)
    print(f"{key} - {y[-1]:.2f}")
    i += 1
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.legend()
plt.ylim(-10000, 410000)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "PMM_comparison_volume_10.pdf"))
plt.show()

In [ ]:
# Set parameters
figsize = (12, 4.5)
xlabel = "Time"
ylabel = "Transaction fees (USDT)"


plt.figure(figsize=figsize)
i = 0
for key, value in results.items():
    x = value["timestamps"]
    y = value["trader_stats"]["cum_costs"]
    label = f"PMM (priority {i})"
    plt.plot(x, y, label=label)
    print(f"{key} - {y[-1]:.2f}")
    i += 1
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.legend()
plt.tight_layout()
# plt.savefig(os.path.join(save_dir, "PMM_comparison_volume_10.pdf"))
plt.show()

In [ ]:
# Compute hitting probability
for key, value in results.items():
    trades = np.array(value["trader_stats"]["trade_count"])
    hits = np.where(trades > 0, 1, 0)
            
    print(f"{key} - {np.mean(hits)*100:.2f}")
    print()

### Comparison of PMM strategies with volume 100 (different SEEDs)

In [ ]:
save_path = "automated_backtests/results_2024-02-25_15-16-39.pickle"

In [ ]:
# Load the results from a pickle file
with open(save_path, "rb") as handle:
    results_pmm = pickle.load(handle)
    
results_pmm.keys()

In [ ]:
# PnL plot 

# Set parameters
figsize = (12, 4.5)
xlabel = "Time"
ylabel = "P&L (USDT)"
avg = []

plt.figure(figsize=figsize)
i = 0
for key, value in results_pmm.items():
    x = value["timestamps"]
    y = value["trader_stats"]["adj_pnl"]
    label = f"PMM (priority {i})"
    plt.plot(x, y, label=label)
    print(f"{key} - {y[-1]:.2f}")
    i += 1
    avg.append(value["trader_stats"]["adj_pnl"][-1])
    
# plt.plot(x, y, color=color)
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.ylim(-60, 210)
# plt.legend()
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "PMM_seeds_comparison_pnl.pdf"))
plt.show()

In [ ]:
print(f"Mean pnl: {np.mean(avg):.2f}")
print(f"Std pnl: {np.std(avg):.2f}")

In [ ]:
# Set parameters
figsize = (12, 4.5)
xlabel = "Time"
ylabel = "Traded volume (USDT)"
avg = []

plt.figure(figsize=figsize)
i = 0
for key, value in results_pmm.items():
    x = value["timestamps"]
    y = value["trader_stats"]["total_volume"]
    label = f"PMM (priority {i})"
    plt.plot(x, y, label=label)
    print(f"{key} - {y[-1]:.2f}")
    avg.append(value["trader_stats"]["total_volume"][-1])
    i += 1
    
plt.xlabel(xlabel)
plt.ylabel(ylabel)
# plt.legend()
plt.ylim(-10000, 230000)
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "PMM_seeds_comparison_volume.pdf"))
plt.show()

In [ ]:
print(f"Mean volume: {np.mean(avg):.2f}")
print(f"Std volume: {np.std(avg):.2f}")

In [ ]:
# Set parameters
figsize = (12, 4.5)
xlabel = "Time"
ylabel = "Transaction fees (USDT)"
avg = []

plt.figure(figsize=figsize)
i = 0
for key, value in results_pmm.items():
    x = value["timestamps"]
    y = value["trader_stats"]["cum_costs"]
    label = f"PMM (priority {i})"
    plt.plot(x, y, label=label)
    print(f"{key} - {y[-1]:.2f}")
    avg.append(value["trader_stats"]["cum_costs"][-1])
    i += 1
    
plt.xlabel(xlabel)
plt.ylabel(ylabel)
# plt.legend()
plt.ylim(-5, 115)
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "PMM_seeds_comparison_fees.pdf"))
plt.show()

In [ ]:
print(f"Mean fees: {np.mean(avg):.2f}")
print(f"Std fees: {np.std(avg):.2f}")

In [ ]:
avg = []

# Compute hitting probability
for key, value in results_pmm.items():
    trades = np.array(value["trader_stats"]["trade_count"])
    hits = np.where(trades > 0, 1, 0)
    avg.append(np.mean(hits) * 100)

In [ ]:
print(f"Mean hitting probability: {np.mean(avg):.2f}%")
print(f"Std hitting probability: {np.std(avg):.2f}")

In [ ]:
results_pmm.keys()

### Comparison of AIRL strategies with volume 100 (different SEEDs)

In [ ]:
save_path = "automated_backtests/results_2024-02-25_21-22-46.pickle"

In [ ]:
# Load the results from a pickle file
with open(save_path, "rb") as handle:
    results_airl = pickle.load(handle)
    
results_airl.keys()

In [ ]:
# PnL plot 

# Set parameters
figsize = (12, 4.5)
xlabel = "Time"
ylabel = "P&L (USDT)"
avg = []

plt.figure(figsize=figsize)
i = 0
for key, value in results_airl.items():
    x = value["timestamps"]
    y = value["trader_stats"]["adj_pnl"]
    label = f"PMM (priority {i})"
    plt.plot(x, y, label=label)
    print(f"{key} - {y[-1]:.2f}")
    i += 1
    avg.append(value["trader_stats"]["adj_pnl"][-1])
    
# plt.plot(x, y, color=color)
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.ylim(-60, 210)
# plt.legend()
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "AIRL_seeds_comparison_pnl.pdf"))
plt.show()

In [ ]:
print(f"Mean pnl: {np.mean(avg):.2f}")
print(f"Std pnl: {np.std(avg):.2f}")

In [ ]:
# Set parameters
figsize = (12, 4.5)
xlabel = "Time"
ylabel = "Traded volume (USDT)"
avg = []

plt.figure(figsize=figsize)
i = 0
for key, value in results_airl.items():
    x = value["timestamps"]
    y = value["trader_stats"]["total_volume"]
    label = f"PMM (priority {i})"
    plt.plot(x, y, label=label)
    print(f"{key} - {y[-1]:.2f}")
    avg.append(value["trader_stats"]["total_volume"][-1])
    i += 1
    
plt.xlabel(xlabel)
plt.ylabel(ylabel)
# plt.legend()
plt.ylim(-10000, 230000)
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "AIRL_seeds_comparison_volume.pdf"))
plt.show()

In [ ]:
print(f"Mean volume: {np.mean(avg):.2f}")
print(f"Std volume: {np.std(avg):.2f}")

In [ ]:
# Set parameters
figsize = (12, 4.5)
xlabel = "Time"
ylabel = "Transaction fees (USDT)"
avg = []

plt.figure(figsize=figsize)
i = 0
for key, value in results_airl.items():
    x = value["timestamps"]
    y = value["trader_stats"]["cum_costs"]
    label = f"PMM (priority {i})"
    plt.plot(x, y, label=label)
    print(f"{key} - {y[-1]:.2f}")
    avg.append(value["trader_stats"]["cum_costs"][-1])
    i += 1
    
plt.xlabel(xlabel)
plt.ylabel(ylabel)
# plt.legend()
plt.ylim(-5, 115)
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "AIRL_seeds_comparison_fees.pdf"))
plt.show()

In [ ]:
print(f"Mean fees: {np.mean(avg):.2f}")
print(f"Std fees: {np.std(avg):.2f}")

In [ ]:
avg = []

# Compute hitting probability
for key, value in results_airl.items():
    trades = np.array(value["trader_stats"]["trade_count"])
    hits = np.where(trades > 0, 1, 0)
    avg.append(np.mean(hits) * 100)

In [ ]:
print(f"Mean hitting probability: {np.mean(avg):.2f}%")
print(f"Std hitting probability: {np.std(avg):.2f}")

### Seed comparison

In [ ]:
# Fix the seed for visualization
seed_visual = 4

In [ ]:
### PMM

# Load the results
ts = results_pmm[f"PMM_prior_1_vol_100_{seed_visual}"]["timestamps"]
trader_stats = results_pmm[f"PMM_prior_1_vol_100_{seed_visual}"]["trader_stats"]
initial_cost = results_pmm[f"PMM_prior_1_vol_100_{seed_visual}"]["initial_cost"]

# Plot the results
# ----------------------------------------------------------------------------
# PLOT - PnL
plt.figure(figsize=(12, 4.5))
plt.plot(ts, trader_stats["adj_pnl"])
plt.xlabel("Time")
plt.ylabel("P&L (USDT)")
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "PMM_seeds_pnl.pdf"))
plt.show()
print(f"Final P&L: {trader_stats['adj_pnl'][-1]}")

# PLOT - Returns
equity = pd.Series(np.array(trader_stats["adj_pnl"]) + initial_cost)
returns = equity.pct_change() * 100
plt.figure(figsize=(12, 4.5))
plt.plot(ts, returns)
plt.xlabel("Time")
plt.ylabel("Returns (%)")
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "PMM_seeds_returns.pdf"))
plt.show()
print("Returns stats")
print(returns.describe())

# PLOT - Drawdowns
dd = drawdowns(equity)
plt.figure(figsize=(12, 4.5))
plt.fill_between(ts, dd, 0, color="red", alpha=0.3)
plt.plot(ts, dd, color="red", alpha=0.5)
plt.xlabel("Time")
plt.ylabel("Drawdown (%)")
plt.ylim(-0.85, 0.05)
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "PMM_seeds_drawdowns.pdf"))
plt.show()
print("Drawdown stats")
print(dd.describe())

# PLOT - Inventory
plt.figure(figsize=(12, 4.5))
plt.plot(ts, trader_stats["inventory"], color="darkorange",)
plt.xlabel("Time")
plt.ylabel("Inventory (SOL)")
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "PMM_seeds_inventory.pdf"))
plt.show()
print("Inventory stats")
print(pd.Series(trader_stats["inventory"]).describe())

# PLOT - Total traded volume
plt.figure(figsize=(12, 4.5))
plt.plot(ts, trader_stats["total_volume"], color="darkorange",)
plt.xlabel("Time")
plt.ylabel("Traded volume (USDT)")
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "PMM_seeds_volume.pdf"))
plt.show()
print("Total volume: ", trader_stats["total_volume"][-1])

# PLOT - Transaction costs
plt.figure(figsize=(12, 4.5))
plt.plot(ts, trader_stats["cum_costs"], color="red",)
plt.xlabel("Time")
plt.ylabel("Transaction fees (USDT)")
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "PMM_seeds_fees.pdf"))
plt.show()
print("Total fees: ", trader_stats["cum_costs"][-1])

In [ ]:
### AIRL 

# Load the results
airl_ts = results_airl[f"RL_prior_1_vol_100_{seed_visual}"]["timestamps"]
airl_trader_stats = results_airl[f"RL_prior_1_vol_100_{seed_visual}"]["trader_stats"]
airl_initial_cost = results_airl[f"RL_prior_1_vol_100_{seed_visual}"]["initial_cost"]

# Plot the results
# ----------------------------------------------------------------------------
# PLOT - PnL
plt.figure(figsize=(12, 4.5))
plt.plot(airl_ts, airl_trader_stats["adj_pnl"], label="AIRL")
plt.plot(ts, trader_stats["adj_pnl"], label="Expert policy")
plt.xlabel("Time")
plt.ylabel("P&L (USDT)")
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.legend(loc="upper left")
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "AIRL_seeds_pnl.pdf"))
plt.show()
print(f"Final P&L: {airl_trader_stats['adj_pnl'][-1]}")

# PLOT - Returns
equity = pd.Series(np.array(airl_trader_stats["adj_pnl"]) + initial_cost)
airl_returns = equity.pct_change() * 100
plt.figure(figsize=(12, 4.5))
plt.plot(airl_ts, airl_returns)
plt.xlabel("Time")
plt.ylabel("Returns (%)")
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "AIRL_seeds_returns.pdf"))
plt.show()
print("Returns stats")
print(returns.describe())

# PLOT - Drawdowns
dd = drawdowns(equity)
plt.figure(figsize=(12, 4.5))
plt.fill_between(ts, dd, 0, color="red", alpha=0.3)
plt.plot(airl_ts, dd, color="red", alpha=0.5)
plt.xlabel("Time")
plt.ylabel("Drawdown (%)")
plt.ylim(-0.85, 0.05)
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "AIRL_seeds_drawdowns.pdf"))
plt.show()
print("Drawdown stats")
print(dd.describe())

# PLOT - Inventory
plt.figure(figsize=(12, 4.5))
plt.plot(airl_ts, airl_trader_stats["inventory"], color="darkorange",)
plt.xlabel("Time")
plt.ylabel("Inventory (SOL)")
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "AIRL_seeds_inventory.pdf"))
plt.show()
print("Inventory stats")
print(pd.Series(airl_trader_stats["inventory"]).describe())

# PLOT - Total traded volume
plt.figure(figsize=(12, 4.5))
plt.plot(airl_ts, airl_trader_stats["total_volume"], label="AIRL")
plt.plot(ts, trader_stats["total_volume"], label="Expert policy")
plt.xlabel("Time")
plt.ylabel("Traded volume (USDT)")
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.legend(loc="upper left")
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "AIRL_seeds_volume.pdf"))
plt.show()
print("Total volume: ", airl_trader_stats["total_volume"][-1])

# PLOT - Transaction costs
plt.figure(figsize=(12, 4.5))
plt.plot(airl_ts, airl_trader_stats["cum_costs"], label="AIRL")
plt.plot(ts, trader_stats["cum_costs"], label="Expert policy")
plt.xlabel("Time")
plt.ylabel("Transaction fees (USDT)")
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.legend(loc="upper left")
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "AIRL_seeds_fees.pdf"))
plt.show()
print("Total fees: ", airl_trader_stats["cum_costs"][-1])